In [101]:
import numpy as np

In [102]:
def fileHandler (fileName):
    # load ascii text and covert to lowercase
    raw_text = open(fileName, 'r', encoding='utf-8').read()
    raw_text = raw_text.lower()
    # create mapping of unique chars to integers
    
    uniqueChars = sorted(list(set(raw_text)))
    vocabSize = len(uniqueChars)
    char_to_int = dict((c, i) for i, c in enumerate(uniqueChars))
    int_to_char = dict((i, c) for i, c in enumerate(uniqueChars))
    
    integers = [char_to_int[char] for char in raw_text]
    #We are doing One hot encoding as We want our MLP to work on it, and a classification problem
    dataMatrix = np.eye(vocabSize)[integers]
    return dataMatrix , vocabSize, char_to_int,int_to_char
    
    

In [103]:
dataMatrix , vocabSize, char_to_int,int_to_char = fileHandler("C:/Users/Lenovo/Desktop/My Git Repo/abcde.txt")

In [104]:
print(dataMatrix.shape,vocabSize,"\n",char_to_int,"\n",int_to_char)

(153594, 7) 7 
 {'\n': 0, ' ': 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6} 
 {0: '\n', 1: ' ', 2: 'a', 3: 'b', 4: 'c', 5: 'd', 6: 'e'}


#### Creating Training Data

In [105]:
# Prepare training input-output pairs
X_train = dataMatrix[:-1]  # All rows except the last one
Y_train = dataMatrix[1:]   # All rows except the first one

inputs = torch.Tensor(X_train)
#Take the max number (1 in our case) indice of the one hot enocding alongside column
labels = torch.Tensor(np.argmax(Y_train, axis=1)) 


In [106]:
print(int_to_char[2])
print(inputs[0])
print(int_to_char)
print()

a
tensor([0., 0., 1., 0., 0., 0., 0.])
{0: '\n', 1: ' ', 2: 'a', 3: 'b', 4: 'c', 5: 'd', 6: 'e'}



In [107]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [108]:
class TextGeneratorMLP1(nn.Module):
    def __init__(self,inputSize,outputSize):
        super().__init__()
        
        self.InputLayer = nn.Linear(in_features=inputSize,out_features=128)
        self.HiddenLayer = nn.Linear(in_features=128,out_features=256)
        self.HiddenLayer2 = nn.Linear(in_features=256,out_features=64)
        self.OutputLayer = nn.Linear(in_features=64,out_features=outputSize)
    
    def forward(self,x):
        x = self.InputLayer(x)
        x = F.relu(x)
        
        x = self.HiddenLayer(x)
        x = F.relu(x)
        
        x = self.HiddenLayer2(x)
        x = F.relu(x)
        
        x = self.OutputLayer(x)
        
        return x
        

In [109]:
model  = TextGeneratorMLP1(vocabSize,vocabSize)

In [110]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [112]:
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    
    optimizer.zero_grad()
    outputs = model(inputs)
    # here no need to use outputs.argmax as criterion CrossEntropyLoss do this and softmax on our behalf
    loss = criterion(outputs, labels.long()) 
    
    loss.backward()
    optimizer.step()
    if (epoch+1) % 50 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    

Epoch [50/1000], Loss: 0.1254
Epoch [100/1000], Loss: 0.1080
Epoch [150/1000], Loss: 0.1069
Epoch [200/1000], Loss: 0.1066
Epoch [250/1000], Loss: 0.1064
Epoch [300/1000], Loss: 0.1063
Epoch [350/1000], Loss: 0.1063
Epoch [400/1000], Loss: 0.1062
Epoch [450/1000], Loss: 0.1062
Epoch [500/1000], Loss: 0.1062
Epoch [550/1000], Loss: 0.1062
Epoch [600/1000], Loss: 0.1062
Epoch [650/1000], Loss: 0.1061
Epoch [700/1000], Loss: 0.1061
Epoch [750/1000], Loss: 0.1061
Epoch [800/1000], Loss: 0.1061
Epoch [850/1000], Loss: 0.1061
Epoch [900/1000], Loss: 0.1061
Epoch [950/1000], Loss: 0.1061
Epoch [1000/1000], Loss: 0.1061


In [115]:
def evaluationFunction():
    model.eval()
    generatedString = ""
    radomStartingPoint = np.random.randint(low=0,high=X_train.shape[0])
    modelInput = inputs[radomStartingPoint]
    print(radomStartingPoint,modelInput)
    for i in range(50):
        output = model(modelInput)
        predicted_index = torch.argmax(output, dim=0).item()
        char = int_to_char[predicted_index]
        generatedString += char
        modelInput = torch.Tensor(np.eye(vocabSize)[char_to_int[char]])
    
    print(generatedString)

In [127]:
evaluationFunction()

134962 tensor([0., 0., 0., 0., 0., 0., 1.])
 abcde abcde abcde abcde abcde abcde abcde abcde a


In [117]:
char_to_int

{'\n': 0, ' ': 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6}

In [118]:
char_to_int

{'\n': 0, ' ': 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6}

In [203]:
dataMatrix , vocabSize, char_to_int,int_to_char = fileHandler("C:/Users/Lenovo/Desktop/My Git Repo/abcde_edcba.txt")

In [219]:
# Prepare training input-output pairs
X_train = np.concatenate((dataMatrix[:-2], dataMatrix[1:-1]), axis=1)  # Concatenate two consecutive one-hot-encoded vectors along columns
Y_train = dataMatrix[2:]  # Next three one-hot-encoded vectors as output

inputs = torch.Tensor(X_train)
#Take the max number (1 in our case) indice of the one hot enocding alongside column
labels = torch.Tensor(np.argmax(Y_train, axis=1)) 

In [229]:
print(dataMatrix[0],dataMatrix[1],dataMatrix[2])

[0. 0. 1. 0. 0. 0. 0.] [0. 0. 0. 1. 0. 0. 0.] [0. 0. 0. 0. 1. 0. 0.]


In [230]:
print(X_train[0],Y_train[0],labels[0].item())

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.] [0. 0. 0. 0. 1. 0. 0.] 4.0


In [235]:
model  = TextGeneratorMLP1(vocabSize*2,vocabSize)

In [236]:
print(model)

TextGeneratorMLP1(
  (InputLayer): Linear(in_features=14, out_features=128, bias=True)
  (HiddenLayer): Linear(in_features=128, out_features=256, bias=True)
  (HiddenLayer2): Linear(in_features=256, out_features=64, bias=True)
  (OutputLayer): Linear(in_features=64, out_features=7, bias=True)
)


In [237]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [238]:
num_epochs = 150
for epoch in range(num_epochs):
    model.train()
    
    optimizer.zero_grad()
    outputs = model(inputs)
    # here no need to use outputs.argmax as criterion CrossEntropyLoss do this and softmax on our behalf
    loss = criterion(outputs, labels.long()) 
    
    loss.backward()
    optimizer.step()
    if (epoch+1) % 50 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [50/150], Loss: 0.1772
Epoch [100/150], Loss: 0.0026
Epoch [150/150], Loss: 0.0010


In [239]:
def evaluationFunction():
    model.eval()
    generatedString = ""
    radomStartingPoint = np.random.randint(low=0,high=X_train.shape[0])
    modelInput = inputs[radomStartingPoint]
    print(radomStartingPoint,modelInput)
    for i in range(50):
        #print(modelInput)
        output = model(modelInput)
        predicted_index = torch.argmax(output, dim=0).item()
        char = int_to_char[predicted_index]
        generatedString += char
        newCharacterEncoding = torch.Tensor(np.eye(vocabSize)[char_to_int[char]])
        modelInput = torch.cat((modelInput[-7:],newCharacterEncoding),dim=0)
        #print(f"NewEncoding:{newCharacterEncoding}, predicted_index:{predicted_index}, char:{char}, modelInput : {modelInput}")
    
    print(generatedString)

In [240]:
evaluationFunction()

39243 tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.])
 edcba
abcde edcba
abcde edcba
abcde edcba
abcde e
